In [10]:
import pandas as pd
import ast
import time
import requests
rubq_test = pd.read_csv('mintaka_test_dates.csv')

In [11]:
rubq_test

,id,question,entities,category,complexityType,answerType,answerLabel,filt,one_hop_rels,subjects,possible_rel,ans,amount,rel_pred,prop_gen,bela_base,bela_base_ents
0,3672da64,In what year did Kaley Cuoco play Cassie Bowde...,"[{'name': 'Q55635222', 'entityType': 'entity',...",movies,generic,date,2020,1.0,NaN,"Q55635222, Q16759",P582,2022-05-26T00:00:00Z,1.0,P580,P571,"[{'offsets': [3, 8, 8, 13, 17, 17, 17, 17, 17,...","Q16759, Q55635222, Q6570684, Q101539, Q15416, ..."
1,1a18c66d,In what year was Edward Furlong born?,"[{'name': 'Q309631', 'entityType': 'entity', '...",movies,generic,date,1977,1.0,NaN,Q309631,P569,1977-08-02T00:00:00Z,1.0,P569,P569,"[{'offsets': [8, 8, 17, 17, 17, 17, 24, 27], '...","Q309631, Q577"
2,e35eecdd,What year was Amy Adams born?,"[{'name': 'Q481832', 'entityType': 'entity', '...",movies,generic,date,1974,1.0,NaN,Q481832,P569,1974-08-20T00:00:00Z,1.0,P569,P569,"[{'offsets': [5, 5, 14, 14, 14, 18], 'lengths'...","Q481832, Q577"
3,7cf65dee,In which year was Mark Hamill born?,"[{'name': 'Q210447', 'entityType': 'entity', '...",movies,generic,date,1951,1.0,NaN,Q210447,P569,1951-09-25T00:00:00Z,1.0,P569,P569,"[{'offsets': [3, 9, 9, 18, 18, 18, 18, 23, 28]...","Q210447, Q577, Q23015077"
4,1a905906,What year did Brad Pitt's acting career begin?,"[{'name': 'Q35332', 'entityType': 'entity', 'l...",movies,generic,date,1987,1.0,NaN,Q35332,P569,1963-12-18T00:00:00Z,1.0,P580,P571,"[{'offsets': [5, 5, 14, 14, 19, 26, 26, 26, 33...","Q35332, Q222749, Q33999, Q282049, Q577"
5,74adcb0b,What year was Betty White born in?,"[{'name': 'Q373895', 'entityType': 'entity', '...",movies,generic,date,1922,1.0,NaN,Q373895,P569,1922-01-17T00:00:00Z,1.0,P569,P569,"[{'offsets': [0, 5, 5, 14, 14, 20], 'lengths':...","Q373895, Q577, Q3186692"
6,7f56e1b9,In what year was the first Jurassic movie rele...,"[{'name': 'Q2336369', 'entityType': 'entity', ...",movies,generic,date,1993,1.0,NaN,Q2336369,P577,1993-06-11T00:00:00Z,1.0,P577,P571,"[{'offsets': [8, 8, 17, 21, 21, 21, 21, 27, 27...","Q2336369, Q167726, Q45805, Q577, Q11424"
7,30cc5bd6,When was the James Bond movie Goldfinger relea...,"[{'name': 'Q106440', 'entityType': 'entity', '...",movies,generic,date,1964,1.0,NaN,"Q106440, Q844",P577,1964-09-17T00:00:00Z,1.0,P577,P9946,"[{'offsets': [13, 13, 13, 13, 19, 19, 24, 30, ...","Q106440, Q844, Q151472"
8,20d55fcc,When was One Hundred and One Dalmatians produced?,"[{'name': 'Q165512', 'entityType': 'entity', '...",movies,generic,date,1961,1.0,NaN,Q165512,P577,1961-01-25T00:00:00Z,1.0,P577,P2754,"[{'offsets': [0, 9, 9, 9, 9, 9, 9, 13, 17, 21,...","Q165512, Q17504"
9,62e016bf,What year was the song Regulate by Warren G. a...,"[{'name': 'Q47159', 'entityType': 'entity', 'l...",music,generic,date,1994,1.0,NaN,"Q47159, Q7309633, Q319585",P569,1969-08-19T00:00:00Z,1.0,P571,P264,"[{'offsets': [18, 23, 23, 23, 23, 29, 35, 35, ...","Q47159, Q319585, Q7309633"


In [12]:
import numpy as np
from tqdm import tqdm

In [13]:
for k in tqdm(range(len(rubq_test))):
    
    inds = np.argsort(ast.literal_eval(rubq_test.loc[k, 'bela_base'])[0]['el_scores'])[::-1]

    ents_sort =[ast.literal_eval(rubq_test.loc[k, 'bela_base'])[0]['entities'][i] for i in inds]
    offsets_sort = [ast.literal_eval(rubq_test.loc[k, 'bela_base'])[0]['offsets'][i] for i in inds]
    lengths_sort = [ast.literal_eval(rubq_test.loc[k, 'bela_base'])[0]['lengths'][i] for i in inds]
    el_scores = [ast.literal_eval(rubq_test.loc[k, 'bela_base'])[0]['el_scores'][i] for i in inds]

    cur_set = set()

    fin_ents = []
    fin_offsets = []
    fin_lengths = []
    fin_el_scores = []

    for j in range(len(ents_sort)):
        if ents_sort[j] not in cur_set:
            cur_set.add(ents_sort[j])
            fin_ents.append(ents_sort[j])
            fin_offsets.append(offsets_sort[j])
            fin_lengths.append(lengths_sort[j])
            fin_el_scores.append(el_scores[j])
            
    start = fin_offsets[0]
    length = fin_lengths[0]
    
    rubq_test.loc[k, 'question_en_rank'] = rubq_test.loc[k, 'question'][:start]+'<e>'+rubq_test.loc[k, 'question'][start+length:]
    rubq_test.loc[k, 'scores_en_rank'] = ", ".join(map(str, fin_el_scores))

100%|██████████| 57/57 [00:00<00:00, 156.17it/s]


In [36]:
# rubq_test.to_csv('rubq_paper_res_sel_rank.csv', index = False)

In [14]:
import requests
import pandas as pd

def return_props(entity_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

      SELECT DISTINCT ?property  WHERE {

        wd:<ENTITY> ?property ?object .}
            """.replace(
                "<ENTITY>", entity_idx)
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    subjects = set()
    for q in range(len(data["results"]['bindings'])):
      if 'http://www.wikidata.org/prop/direct/' in data["results"]['bindings'][q]['property']['value']:
        subjects.add(data["results"]['bindings'][q]['property']['value'].split('http://www.wikidata.org/prop/direct/')[-1])


    subj_list = list(subjects)

    return subj_list

In [41]:
ents = rubq_test.loc[1, 'bela_base_ents'].split(', ')
ents

['Q2222', 'Q8261', 'Q7882782', 'Q5783996', 'Q2035679']

In [15]:
rubq_test['ids_one_hops_dict_en'] = np.nan

In [16]:
for j in tqdm(range(len(rubq_test))):
    if pd.isna(rubq_test.loc[j, 'ids_one_hops_dict_en']):
        ents = rubq_test.loc[j, 'bela_base_ents'].split(', ')
        id_rels = dict()
        for i in range(len(ents)):
            try:
                time.sleep(1)
                cur_rel = ', '.join(return_props(ents[i]))
                id_rels[ents[i]] = cur_rel
            except:
                time.sleep(1)
                continue 
        rubq_test.loc[j, 'ids_one_hops_dict_en'] = str(id_rels)

100%|██████████| 57/57 [03:13<00:00,  3.39s/it]


In [9]:
rubq_test.to_csv('lcquad_simple.csv', index = False)

In [67]:
rubq_test.to_csv('rubq_paper_res_sel_rank.csv', index = False)

In [69]:
mintaka = pd.read_csv('mintaka_paper_res_sel_rank.csv')

In [71]:
mintaka.columns

Index(['id', 'question', 'entities', 'category', 'complexityType',
       'answerType', 'answerLabel', 'questionType', 'ents_lbls', 'ents_len',
       'bela_base', 'bela_base_ents', 'mgenre_with_bela_en_ques',
       'pred_with_bela_ner_en', 'scores_with_bela_ner_en', 'pred_relation',
       'pred_prop_fast', 'pred_obj_fall_init_en', 'pred_obj_fall_init',
       'pred_obj_fast', 'pred_fin', 'pred_label_dbloss', 'max_pred_id',
       'ent_lbl', 'ner_init', 'question_ner_init', 'question_with_mintaka_ner',
       'pred_mgenre_with_spacy_ner', 'pred_mgenre_spacy_ner',
       'scores_mgenre_spacy_ner', 'ents_falcon', 'rels_falcon', 'one_hop_rels',
       'question_en_rank', 'scores_en_rank'],
      dtype='object')

In [72]:
mintaka['ids_one_hops_dict_en'] = np.nan

In [75]:
for j in tqdm(range(len(mintaka))):
    if pd.isna(mintaka.loc[j, 'ids_one_hops_dict_en']):
        ents = mintaka.loc[j, 'bela_base_ents'].split(', ')
        id_rels = dict()
        for i in range(len(ents)):
            try:
                time.sleep(1)
                cur_rel = ', '.join(return_props(ents[i]))
                id_rels[ents[i]] = cur_rel
            except:
                time.sleep(1)
                continue 
        mintaka.loc[j, 'ids_one_hops_dict_en'] = str(id_rels)

100%|██████████| 340/340 [25:50<00:00,  4.56s/it]


In [77]:
mintaka.to_csv('mintaka_paper_res_sel_rank.csv', index = False)

In [78]:
wdsq = pd.read_csv('wdsq_paper_res_sel_test.csv')

In [79]:
wdsq.columns

Index(['subject', 'property', 'object', 'question',
       'pred_on_wdsq_for_wdsq_ner', 'pred_relation_upd', 'pred_obj',
       'expected_ans', 'real_ans', 'object_gold', 'object_gold_v2',
       'pred_relation', 'm3m_sel'],
      dtype='object')

In [80]:
wdsq['ids_one_hops_dict_en'] = np.nan

In [83]:
for j in tqdm(range(len(wdsq))):
    if pd.isna(wdsq.loc[j, 'ids_one_hops_dict_en']):
        ents = wdsq.loc[j, 'pred_on_wdsq_for_wdsq_ner'].split(', ')
        id_rels = dict()
        for i in range(len(ents)):
            try:
                time.sleep(1)
                cur_rel = ', '.join(return_props(ents[i]))
                id_rels[ents[i]] = cur_rel
            except:
                time.sleep(1)
                continue 
        wdsq.loc[j, 'ids_one_hops_dict_en'] = str(id_rels)

 76%|███████▌  | 4274/5622 [4:27:26<1:26:14,  3.84s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [87]:
wdsq.query('ids_one_hops_dict_en.isna()')

,subject,property,object,question,pred_on_wdsq_for_wdsq_ner,pred_relation_upd,pred_obj,expected_ans,real_ans,object_gold,object_gold_v2,pred_relation,m3m_sel,ids_one_hops_dict_en


In [88]:
wdsq.to_csv('wdsq_paper_res_sel_test.csv', index = False)

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
def run_query_object(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   wd:<ENTITY> wdt:<PROPERTY> ?object .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [4]:
mintaka = pd.read_csv('mintaka_paper_res_sel_rank.csv')

In [5]:
mintaka.columns

Index(['id', 'question', 'entities', 'category', 'complexityType',
       'answerType', 'answerLabel', 'questionType', 'ents_lbls', 'ents_len',
       'bela_base', 'bela_base_ents', 'mgenre_with_bela_en_ques',
       'pred_with_bela_ner_en', 'scores_with_bela_ner_en', 'pred_relation',
       'pred_prop_fast', 'pred_obj_fall_init_en', 'pred_obj_fall_init',
       'pred_obj_fast', 'pred_fin', 'pred_label_dbloss', 'max_pred_id',
       'ent_lbl', 'ner_init', 'question_ner_init', 'question_with_mintaka_ner',
       'pred_mgenre_with_spacy_ner', 'pred_mgenre_spacy_ner',
       'scores_mgenre_spacy_ner', 'ents_falcon', 'rels_falcon', 'one_hop_rels',
       'question_en_rank', 'scores_en_rank', 'ids_one_hops_dict_en',
       'subj_rank', 'rel_rank', 'rel_rank_id'],
      dtype='object')

In [9]:
for i in tqdm(range(len(mintaka))):
    subj = mintaka.loc[i, 'subj_rank']
    rel = mintaka.loc[i, 'rel_rank_id']
    try:
        time.sleep(1)
        ans = run_query_object(subj, rel)
        mintaka.loc[i, 'ans_rank'] = ans
    except:
        continue

100%|██████████| 340/340 [07:57<00:00,  1.41s/it]


In [24]:
for j in tqdm(range(len(mintaka))):
    cands = mintaka.loc[j, 'ans_rank'].split(', ')
    elems = []
    for k in range(len(cands)):
        if 'entity/' in cands[k]:
            elems.append(cands[k].split('entity/')[1])
        else:
            elems.append(cands[k])
    mintaka.loc[j, 'ans_rank_clean'] = ', '.join(elems)

100%|██████████| 340/340 [00:00<00:00, 5124.91it/s]


In [33]:
mintaka.columns

Index(['id', 'question', 'entities', 'category', 'complexityType',
       'answerType', 'answerLabel', 'questionType', 'ents_lbls', 'ents_len',
       'bela_base', 'bela_base_ents', 'mgenre_with_bela_en_ques',
       'pred_with_bela_ner_en', 'scores_with_bela_ner_en', 'pred_relation',
       'pred_prop_fast', 'pred_obj_fall_init_en', 'pred_obj_fall_init',
       'pred_obj_fast', 'pred_fin', 'pred_label_dbloss', 'max_pred_id',
       'ent_lbl', 'ner_init', 'question_ner_init', 'question_with_mintaka_ner',
       'pred_mgenre_with_spacy_ner', 'pred_mgenre_spacy_ner',
       'scores_mgenre_spacy_ner', 'ents_falcon', 'rels_falcon', 'one_hop_rels',
       'question_en_rank', 'scores_en_rank', 'ids_one_hops_dict_en',
       'subj_rank', 'rel_rank', 'rel_rank_id', 'ans_rank', 'ans_rank_clean'],
      dtype='object')

In [28]:
count = 0
for i in range(len(mintaka)):
    ans = mintaka.loc[i, 'ans_rank_clean'].split(', ')[0]
    if ans == mintaka.loc[i, 'answerLabel']:
        count += 1

In [30]:
print(count/len(mintaka))

0.2852941176470588


In [38]:
mintaka.columns

Index(['id', 'question', 'entities', 'category', 'complexityType',
       'answerType', 'answerLabel', 'questionType', 'ents_lbls', 'ents_len',
       'bela_base', 'bela_base_ents', 'mgenre_with_bela_en_ques',
       'pred_with_bela_ner_en', 'scores_with_bela_ner_en', 'pred_relation',
       'pred_prop_fast', 'pred_obj_fall_init_en', 'pred_obj_fall_init',
       'pred_obj_fast', 'pred_fin', 'pred_label_dbloss', 'max_pred_id',
       'ent_lbl', 'ner_init', 'question_ner_init', 'question_with_mintaka_ner',
       'pred_mgenre_with_spacy_ner', 'pred_mgenre_spacy_ner',
       'scores_mgenre_spacy_ner', 'ents_falcon', 'rels_falcon', 'one_hop_rels',
       'question_en_rank', 'scores_en_rank', 'ids_one_hops_dict_en',
       'subj_rank', 'rel_rank', 'rel_rank_id', 'ans_rank', 'ans_rank_clean'],
      dtype='object')

In [40]:
def run_query_subject(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

  ?subject wdt:<PROPERTY> wd:<ENTITY> .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['subject']['value'])                 
    return ', '.join(objects)

In [41]:
mintaka['pred_obj_maxpred'] = np.nan

In [42]:
for k in tqdm(range(len(mintaka))):
    if pd.isna(mintaka.loc[k, 'pred_obj_maxpred']) and pd.notna(mintaka.loc[k, 'max_pred_id']):
        try:
            time.sleep(1)
            cur_subjects = mintaka.loc[k, 'bela_base_ents'].split(', ')
            for w in range(len(cur_subjects)):
                cur_subj = mintaka.loc[k, 'bela_base_ents'].split(', ')[w]
                cur_rel = mintaka.loc[k, 'max_pred_id'].split(', ')[0]
                if cur_rel[0] == 'P':
                    obj_pred = run_query_object(cur_subj, cur_rel)
                    if len(obj_pred) > 0:
                        mintaka.loc[k, 'pred_obj_maxpred'] = obj_pred
                        break
                else:
                    time.sleep(1)
                    cur_rel = 'P'+cur_rel[1:]
                    obj_pred = run_query_subject(cur_subj, cur_rel)
                    if len(obj_pred) > 0:
                        mintaka.loc[k, 'pred_obj_maxpred'] = obj_pred
                        break
        except:
            time.sleep(1)
            continue

100%|██████████| 340/340 [08:34<00:00,  1.51s/it]


In [44]:
for j in tqdm(range(len(mintaka))):
    if pd.notna(mintaka.loc[j, 'pred_obj_maxpred']):
        cands = mintaka.loc[j, 'pred_obj_maxpred'].split(', ')
        elems = []
        for k in range(len(cands)):
            if 'entity/' in cands[k]:
                elems.append(cands[k].split('entity/')[1])
            else:
                elems.append(cands[k])
        mintaka.loc[j, 'ans_class_rank'] = ', '.join(elems)
    else:
        mintaka.loc[j, 'ans_class_rank'] = mintaka.loc[j, 'ans_rank_clean']

100%|██████████| 340/340 [00:00<00:00, 4577.33it/s]


In [46]:
count = 0
for i in range(len(mintaka)):
    ans = mintaka.loc[i, 'ans_rank_clean'].split(', ')[0]
    if ans == mintaka.loc[i, 'answerLabel']:
        count += 1
print(count/len(mintaka))    

0.2852941176470588


In [48]:
mintaka.to_csv('mintaka_paper_res_sel_rank.csv', index = False)